# Optimizing over trained graph neural networks

This notebook explains how OMLT is used to optimize over trained graph neural networks (GNNs).

**NOTE:** For simplicity, we skip the training process and just use random parameters for GNNs.


## Library Setup

This notebook assumes you have a working PyTorch environment to define a Dense NN. This Dense NN is then formulated in Pyomo using OMLT which therefore requires working Pyomo and OMLT installations.

The required Python libraries used in this notebook are as follows:

- `numpy`: used for transformation of parameters

- `torch`: the machine learning language used for neural networks

- `torch_geometric`: the machine learning language used for graph neural networks

- `pyomo`: the algebraic modeling language for Python, it is used to define the optimization model passed to the solver

- `onnx`: used to express trained neural network models

- `omlt`: the package this notebook demonstrates. OMLT can formulate machine learning (such as neural networks) within Pyomo

**NOTE:** This notebook also assumes you have a working MIP solver executable to solve optimization problems in Pyomo. The open-source solver CBC is called by default. 


## Example 1: Optimizing a GNN with Fixed Graph

Define a GCN in `torch_geometric` as follows:

In [ ]:
import numpy as np
import pyomo.environ as pyo
import torch
from torch.nn import Linear, ReLU, Sigmoid
from torch_geometric.nn import GCNConv, Sequential, global_mean_pool

from omlt import OmltBlock
from omlt.io.torch_geometric import gnn_with_fixed_graph


def GCN_Sequential(activation, pooling):
    torch.manual_seed(123)
    return Sequential(
        "x, edge_index, batch",
        [
            (GCNConv(2, 4), "x, edge_index -> x"),
            activation(),
            (GCNConv(4, 4), "x, edge_index -> x"),
            activation(),
            Linear(4, 4),
            (pooling, "x, batch -> x"),
            Linear(4, 2),
            activation(),
            Linear(2, 1),
        ],
    )

This model has two types of `Linear` layers: the first linear layer maps in-features to out-features for each node, the last two linear layers map features after pooling. For illustration purposes, we use `load_torch_geometric_sequential` to show the transformed model in OMLT (this step is not needed for later formulation):

In [2]:
from omlt.io.torch_geometric import load_torch_geometric_sequential

# define a GCN sequential model
nn = GCN_Sequential(ReLU, global_mean_pool)
# number of nodes
N = 3
# adjacency matrix
A = np.array([[1, 1, 0], [1, 1, 1], [0, 1, 1]])

# load the model into OMLT
net = load_torch_geometric_sequential(nn, N, A)

for layer_id, layer in enumerate(net.layers):
    print(f"{layer_id}\t{layer}\t{layer.activation}")

0	InputLayer(input_size=[6], output_size=[6])	linear
1	GNNLayer(input_size=[6], output_size=[12])	relu
2	GNNLayer(input_size=[12], output_size=[12])	relu
3	DenseLayer(input_size=[12], output_size=[12])	linear
4	DenseLayer(input_size=[12], output_size=[4])	linear
5	DenseLayer(input_size=[4], output_size=[2])	relu
6	DenseLayer(input_size=[2], output_size=[1])	linear


Two GCN layers are rewritten into two `GNNLayer` in OMLT given $N$ and $A$. The first linear layer is expanded since it maps features of each node. The pooling layer is equivalently transformed into a `DenseLayer`. The last two linear layers are the same as before since features of each node are pooled.

Besides giving $N$ and $A$, one needs to define bounds for inputs:

In [3]:
# define a GCN sequential model
nn1 = GCN_Sequential(ReLU, global_mean_pool)
# number of nodes
N = 3
# adjacency matrix
A = np.array([[1, 1, 0], [1, 1, 1], [0, 1, 1]])

# size of inputs = number of nodes x number of input features
input_size = [6]
# define lower and upper bounds for each input
input_bounds = {}
for i in range(input_size[0]):
    input_bounds[(i)] = (-1.0, 1.0)

After having these information, the last step is to create an `OmltBlock` and build formulation in this block using `gnn_with_fixed_graph`:

In [4]:
# create pyomo model
m1 = pyo.ConcreteModel()

# create an OMLT block for the neural network and build its formulation
m1.nn = OmltBlock()

# build formulation in block m.nn
gnn_with_fixed_graph(m1.nn, nn1, N, A, scaled_input_bounds=input_bounds)

# set the objective as the single output of the model
m1.obj = pyo.Objective(expr=m1.nn.outputs[0])

# solve the optimization problem
status = pyo.SolverFactory("cbc").solve(m1, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Oct 13 2018 

command line - /rds/general/user/sz421/home/anaconda3/envs/OMLT_test/bin/cbc -printingOptions all -import /var/tmp/pbs.377756.pbs/tmponfhl_r6.pyomo.lp -stat=1 -solve -solu /var/tmp/pbs.377756.pbs/tmponfhl_r6.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 172 (-222) rows, 111 (-75) columns and 608 (-267) elements
Statistics for presolved model
Original problem has 26 integers (26 of which binary)
Presolved problem has 25 integers (25 of which binary)
==== 110 zero objective 2 different
1 variables have objective of -0.0421598
110 variables have objective of 0
==== absolute objective values 2 different
110 variables have objective of 0
1 variables have objective of 0.0421598
==== for integers 25 zero objective 1 different
25 variables have objective of 0
==== for integers absolute objective values 1 different
25 variables have objective of 0
===== end objective counts

We can evaluate the solution in original model to verify it:

In [5]:
X = []
edges = []
for u in range(N):
    for v in range(N):
        if u != v and pyo.value(m1.nn.A[u, v]):
            edges.append((u, v))
for i in range(6):
    X.append(pyo.value(m1.nn.inputs[i]))
X = np.array(X).reshape(3, 2)
edges = np.transpose(np.array(edges)).reshape(2, -1)
nn1.eval()
print(nn1(torch.tensor(X).float(), torch.tensor(edges).long(), None).detach().numpy())

[[0.31796885]]


For smooth activation function like Sigmoid, a smooth optimization solvers (such as Ipopt) is needed:

In [6]:
# define a GCN sequential model
nn2 = GCN_Sequential(Sigmoid, global_mean_pool)

# create pyomo model
m2 = pyo.ConcreteModel()

# create an OMLT block for the neural network and build its formulation
m2.nn = OmltBlock()

# build formulation in block m.nn
gnn_with_fixed_graph(m2.nn, nn2, N, A, scaled_input_bounds=input_bounds)

# set the objective as the single output of the model
m2.obj = pyo.Objective(expr=m2.nn.outputs[0])

# solve the optimization problem
status = pyo.SolverFactory("ipopt").solve(m2, tee=True)

Ipopt 3.14.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.12, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:      395
Number of nonzeros in inequality constraint Jacobian.:      276
Number of nonzeros in Lagrangian Hessian.............:       26

Total number of variables............................:      148
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      146
                     variables with only upper bounds:        0
Total number of equality constraints.................:      100
Total number

## Example 2: Optimizing a GNN with Non-Fixed Graph

Since GCN is not supported when the input graph is not fixed, we define a SAGE in `torch_geometric` as follows:

In [ ]:
import numpy as np
import pyomo.environ as pyo
import torch
from torch.nn import ReLU
from torch_geometric.nn import SAGEConv, global_add_pool

from omlt import OmltBlock
from omlt.io.torch_geometric import gnn_with_non_fixed_graph


def SAGE_Sequential(activation, pooling):
    torch.manual_seed(123)
    return Sequential(
        "x, edge_index, batch",
        [
            (SAGEConv(2, 4, aggr="sum"), "x, edge_index -> x"),
            activation(),
            (SAGEConv(4, 4, aggr="sum"), "x, edge_index -> x"),
            activation(),
            Linear(4, 4),
            (pooling, "x, batch -> x"),
            Linear(4, 2),
            activation(),
            Linear(2, 1),
        ],
    )

We follow the same procedure as in Example 1 except that $A$ is no longer needed for `gnn_with_non_fixed_graph`:

In [8]:
# define a GAGE sequential model
nn3 = SAGE_Sequential(ReLU, global_add_pool)
# number of nodes
N = 3

# size of inputs = number of nodes x number of input features
input_size = [6]
# define lower and upper bounds for each input
input_bounds = {}
for i in range(input_size[0]):
    input_bounds[(i)] = (-1.0, 1.0)

# create pyomo model
m3 = pyo.ConcreteModel()

# create an OMLT block for the neural network and build its formulation
m3.nn = OmltBlock()

# build formulation in block m.nn
gnn_with_non_fixed_graph(m3.nn, nn3, N, scaled_input_bounds=input_bounds)

# set the objective as the single output of the model
m3.obj = pyo.Objective(expr=m3.nn.outputs[0])

# solve the optimization problem
status = pyo.SolverFactory("cbc").solve(m3, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Oct 13 2018 

command line - /rds/general/user/sz421/home/anaconda3/envs/OMLT_test/bin/cbc -printingOptions all -import /var/tmp/pbs.377756.pbs/tmpfd6tc67i.pyomo.lp -stat=1 -solve -solu /var/tmp/pbs.377756.pbs/tmpfd6tc67i.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 260 (-137) rows, 141 (-51) columns and 852 (-173) elements
Statistics for presolved model
Original problem has 32 integers (32 of which binary)
Presolved problem has 29 integers (29 of which binary)
==== 139 zero objective 3 different
139 variables have objective of 0
1 variables have objective of 0.203177
1 variables have objective of 0.686721
==== absolute objective values 3 different
139 variables have objective of 0
1 variables have objective of 0.203177
1 variables have objective of 0.686721
==== for integers 29 zero objective 1 different
29 variables have objective of 0
==== for integers absolute objective val